# Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# install

In [ ]:
!pip3 install transformers peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manyl

# 7.2.1 LoRA

### LoRA 실습

In [ ]:
from transformers import AutoModelForCausalLM, AutoConfig
from peft import get_peft_model, LoraConfig, TaskType

# 모델 로드
model = AutoModelForCausalLM.from_pretrained("beomi/gemma-ko-2b")

# peft
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# 모델 저장
model.save_pretrained("/content/drive/MyDrive/Books/outputs/peft")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 921,600 || all params: 2,507,094,016 || trainable%: 0.036759690467068624


In [ ]:
!ls /content/drive/MyDrive/Books/outputs/peft
!du -sh /content/drive/MyDrive/Books/outputs/peft

adapter_config.json  adapter_model.safetensors	README.md
3.6M	/content/drive/MyDrive/Books/outputs/peft


### 모델 추론

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import PeftModel, PeftConfig

peft_model_id = "/content/drive/MyDrive/Books/outputs/peft"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)
model = model.to('cuda')
model.eval()

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
inputs = tokenizer("대한민국 보물 제 1호는 무엇입니까? ", return_tensors="pt")

gen_cfg = GenerationConfig(
    do_sample=True,
    min_length=10,
    max_length=50,
    penalty_alpha=0.8,
    repetition_penalty=1.5,
    no_repeat_ngram_size=3,
    temperature=0.8,
    top_k=50,
    top_p=0.92,
)
with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), generation_config=gen_cfg)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

대한민국 보물 제 1호는 무엇입니까? 20. 조선시대의 건축 양식 중, 수원지역에서 많이 사용된 것은 ?


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

model = AutoModelForCausalLM.from_pretrained("nlpai-lab/ko-gemma-2b-v1")
model = model.to('cuda')
model.eval()

tokenizer = AutoTokenizer.from_pretrained("nlpai-lab/ko-gemma-2b-v1")
inputs = tokenizer.apply_chat_template(
    [
        {"role": "user", "content": "대한민국 보물 제 1호는 무엇입니까?"},
    ],
    add_generation_prompt=True,
    return_tensors="pt",
)

gen_cfg = GenerationConfig(
    do_sample=True,
    min_length=10,
    max_length=50,
    penalty_alpha=0.8,
    # repetition_penalty=1.5,
    # no_repeat_ngram_size=3,
    # temperature=0.8,
    # top_k=50,
    # top_p=0.92,
)
with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), generation_config=gen_cfg)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]